# Audio classification architectures

Ses sınıflandırmasının amacı, bir ses girişi için bir sınıf etiketi tahmin etmektir. Model, tüm giriş dizisini kapsayan tek bir sınıf etiketini tahmin edebilir veya her ses karesi için (tipik olarak giriş sesinin her 20 milisaniyesinde bir) bir etiket tahmin edebilir; bu durumda modelin çıktısı bir sınıf etiketi olasılıkları dizisidir. İlkine örnek olarak belirli bir sesi hangi kuşun çıkardığını tespit etmek verilebilir; ikincisine örnek olarak ise modelin herhangi bir anda hangi konuşmacının konuştuğunu tahmin ettiği konuşmacı günlüğü oluşturma verilebilir.

## Classification using spectrograms

Ses sınıflandırması yapmanın en kolay yollarından biri, bunun bir görüntü sınıflandırma problemi olduğunu varsaymaktır!

Bir spektrogramın iki boyutlu bir şekil tensörü (frekanslar, dizi uzunluğu) olduğunu hatırlayın. Ses verileriyle ilgili bölümde bu spektrogramları görüntü olarak çizmiştik. Bilin bakalım ne oldu? Spektrogramı tam anlamıyla bir görüntü olarak ele alabilir ve ResNet gibi normal bir CNN sınıflandırıcı modeline aktarabilir ve çok iyi tahminler elde edebiliriz. Daha da iyisi, ViT gibi bir görüntü dönüştürücü model kullanabiliriz.

Audio Spectrogram Transformer'ın yaptığı budur. ViT veya Vision Transformer modelini kullanır ve normal görüntüler yerine spektrogramları girdi olarak geçirir. Dönüştürücünün kendi kendine dikkat katmanları sayesinde, model küresel bağlamı bir CNN'den daha iyi yakalayabilir.

**Tıpkı ViT gibi AST modeli de ses spektrogramını 16×16 piksellik kısmen örtüşen görüntü yamaları dizisine böler**. Bu yamalar dizisi daha sonra bir gömme dizisine yansıtılır ve bunlar her zamanki gibi dönüştürücü kodlayıcıya girdi olarak verilir. AST sadece bir kodlayıcı dönüştürücü modelidir ve bu nedenle çıktı, her 16×16 giriş yaması için bir tane olmak üzere bir dizi gizli durumdur. Bunun üzerinde, gizli durumları sınıflandırma olasılıklarına eşlemek için sigmoid aktivasyonlu basit bir sınıflandırma katmanı bulunur.

![](https://huggingface.co/datasets/huggingface-course/audio-course-images/resolve/main/ast.png)

## Any transformer can be a classifier

Önceki bölümde CTC'nin sadece kodlayıcı dönüştürücü kullanarak otomatik konuşma tanıma gerçekleştirmek için etkili bir teknik olduğunu görmüştünüz. Bu tür CTC modelleri zaten sınıflandırıcıdır ve bir tokenizer kelime dağarcığından sınıf etiketleri için olasılıkları tahmin eder. Bir CTC modelini alıp etiketleri değiştirerek ve özel CTC kaybı yerine normal bir çapraz entropi kayıp fonksiyonu ile eğiterek genel amaçlı bir ses sınıflandırıcısına dönüştürebiliriz.

Örneğin, HF Transformers bir Wav2Vec2ForCTC modelinin yanı sıra Wav2Vec2ForSequenceClassification ve Wav2Vec2ForAudioFrameClassification modellerine de sahiptir. Bu modellerin mimarileri arasındaki tek fark, sınıflandırma katmanının boyutu ve kullanılan kayıp fonksiyonudur.

Aslında, herhangi bir sadece kodlayıcı ses dönüştürücü modeli, gizli durumlar dizisinin üzerine bir sınıflandırma katmanı eklenerek bir ses sınıflandırıcıya dönüştürülebilir. (Sınıflandırıcıların genellikle bir dönüştürücü kod çözücüye ihtiyacı yoktur).

Tüm sekans için tek bir sınıflandırma puanı tahmin etmek için (Wav2Vec2ForSequenceClassification), model gizli durumların ortalamasını alır ve bunu sınıflandırma katmanına besler. Çıktı tek bir olasılık dağılımıdır.

Her ses karesi için ayrı bir sınıflandırma yapmak için (Wav2Vec2ForAudioFrameClassification), sınıflandırıcı gizli durumlar dizisi üzerinde çalıştırılır ve böylece sınıflandırıcının çıktısı da bir dizidir.